In [5]:
import sys
import os
import json
import datetime
import numpy as np
import pandas as pd
import pymysql
import statsmodels.api as sm
import statsmodels.formula.api as smf
import predictions as pr
import cost

def read_in():
    if sys.stdin.isatty()== True:
        return({'startdate':datetime.date.today(),'year':15,'type':'month','size':99,'weight':1.5,'average_time':3.4})
    if sys.stdin.isatty()== False:
        lines=sys.stdin.readlines()
        parsed=json.loads(lines[0])
        return(parsed)

In [6]:
params=read_in()
params['year']=int(params['year']);params['size']=float(params['size']);params['weight']=float(params['weight']);params['averagetime']=float(params['averagetime']);
cost=cost.total_cost_calc(params)
revenue=pr.predict(params['scenario'],params['startdate'],params['year'])
revenue['rec_price']=100
revenue['smp_revenue']=revenue['smp_price']*30*params['weight']*params['size']*params['averagetime']
revenue['rec_revenue']=revenue['rec_price']*30*params['weight']*params['size']*params['averagetime']
revenue.index=pd.period_range(start=revenue.index[0],periods=revenue.shape[0],freq="m")
result=pd.concat([revenue,cost],axis=1)
result["date"]=result.index
result["smp_revenue"]=result["smp_revenue"].astype(int)
result["rec_revenue"]=result["rec_revenue"].astype(int)
result

IndexError: list index out of range

In [15]:
x=pd.period_range(start="2014-01-10",periods=100,freq="m")
x.day

Int64Index([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31,
            30, 31, 31, 30, 31, 30, 31, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31,
            30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31,
            30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31,
            30, 31, 30, 31, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31,
            28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30],
           dtype='int64')